## Optimizing MGA-1dsm



In [ ]:
%matplotlib notebook

In [ ]:
import pykep as pk
import pygmo as pg
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

import gtocprep as g

In [ ]:
V0 = 3000 # m/s
R0 = 1000 * g.RADIUS_SATURN # m
M0 = 5000 # kg
ISP = 308 # s
MPROP = 2000 # kg
T_START, T_END = 20000, 24000 # MJD2000

In [ ]:
# Generate pseudo-planet with circular orbit as starting point
a = R0
e = 0.0
# we take the remaining orbital elements from Titan
i, W, w, M = 2.769808447473740E+01 * pk.DEG2RAD, 1.690711523205395E+02 * pk.DEG2RAD, 1.746196276345883E+02 * pk.DEG2RAD, 1.869169041332501E+02 * pk.DEG2RAD
pseudo = pk.planet.keplerian(g.REF_EPOCH, [a,e,i,W,w,M], g.MU_SATURN, 1.0, 1.0, 1.1, 'pseudo')

In [ ]:
seq = [pseudo, g.titan, g.titan, g.titan]

# good starting epoch: 23948.880338224088
t0 = [pk.epoch(23947), pk.epoch(23949)]

# tof = [[215.0, 215.5], [50, 100], [20, 40]]
tof = [300, 365]

vinf = [3, 3] # m/s

In [ ]:
udp = pk.trajopt.mga_1dsm(seq, t0, tof, vinf)
# udp = g.mga_1dsm_incipit(seq, t0, tof, vinf)

prob = pg.problem(udp)
uda = pg.algorithms.sade(memory=True)
alg = pg.algorithm(uda)
pop = pg.population(prob, 20)

In [ ]:
for i in tqdm(list(range(1000))):
    pop = alg.evolve(pop)
dv = pop.champion_f[0]
veff = ISP * pk.G0
mf = M0 / np.exp(dv / veff)
print('DV: {0:0.3f}m/s, propellant used: {1:0.3f}kg'.format(dv, M0 - mf))

In [ ]:
udp.pretty(pop.champion_x)

In [ ]:
ax = udp.plot(pop.champion_x)
axlim = .02
ax.set_xlim3d(-axlim, axlim)
ax.set_ylim3d(-axlim, axlim)
ax.set_zlim3d(-axlim, axlim)
ax.view_init(elev=90, azim=90)
plt.show()